# Caltech Machine Learning Homework # 2

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import random
import math
import matplotlib.pyplot as plt
from typing import List

Instructions: https://work.caltech.edu/homework/hw3.pdf

Answers: http://work.caltech.edu/homework/hw3_sol.pdf

## Generalization Error

![](imgs/generr1.png)

The bound should be

M = 1
epsilon = 0.05

2 * M * exp(-2 * (epsilon**2) * N) = 0.03

![](imgs/math1.jpg)

So the minimum number of N needed should be (b)

### 2. Repeat for the case M = 10

![](imgs/math2.jpg)

So

In [3]:
M = 10
N_bound = (np.log(0.03) - np.log(2) - np.log(M)) / (-2 * (0.05**2))
N_bound

1300.4580341747944

So the answer here is (c)

### 3. Repeat for the case M = 100

In [6]:
M = 100
N_bound = (np.log(0.03) - np.log(2) - np.log(M)) / (-2 * (0.05**2))
N_bound

1760.9750527736037

So the answer here is (d)